In [11]:
# Common 
import os
import keras
import numpy as np 
import tensorflow as tf

# Data
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

# Data Visualization 
import plotly.express as px

# Models
from keras import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet101V2

In [3]:
info_file = './monkeypox-skin-lesion-dataset/Monkeypox_Dataset_metadata.csv'
info = pd.read_csv(info_file)
info.head()

,ImageID,Label
0,M01_01,Monkeypox
1,M01_02,Monkeypox
2,M01_03,Monkeypox
3,M01_04,Monkeypox
4,M02_01,Monkeypox


In [4]:
fig = px.pie(
    names=info.Label.unique(),
    values=info.Label.value_counts(),
    hole=0.2,
    width=500,
    height=500
)
fig.update_layout({'title':{'text':'Class Distribution','x':0.45}})
fig.show()

In [5]:
# Data Generators
train_gen = ImageDataGenerator(rescale=1./255)
valid_gen = ImageDataGenerator(rescale=1./255,validation_split=0.4)

# Loading Data
train_data = train_gen.flow_from_directory("./monkeypox-skin-lesion-dataset/Augmented Images/Augmented Images",target_size=(256,256),shuffle=True,class_mode='binary')
valid_data = valid_gen.flow_from_directory('./monkeypox-skin-lesion-dataset/Original Images/Original Images',target_size=(256,256),shuffle=True,subset='training',class_mode='binary')
test_data = valid_gen.flow_from_directory('./monkeypox-skin-lesion-dataset/Original Images/Original Images',target_size=(256,256),shuffle=True,subset='validation',class_mode='binary')

Found 3192 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [12]:
with tf.device('/GPU:0'):
  base_model = ResNet101V2(
      include_top=False,
      input_shape=(256,256,3)
  )
  base_model.trainable = False

  model = Sequential([
      base_model,
      GlobalAveragePooling2D(),
      Dense(256,activation='relu'),
      BatchNormalization(),
      Dense(164,activation='relu'),
      BatchNormalization(),
      Dense(1,activation='sigmoid')
  ])

  model.compile(
      loss='binary_crossentropy',
      optimizer='Adam',
      metrics=['accuracy']
  )

  cb = [EarlyStopping(patience=5,monitor='val_accuracy',mode='max',restore_best_weights=True),ModelCheckpoint("ResNet101V2-01.h5",save_best_only=True)]

  model.fit(
      train_data,
      epochs=50,
      validation_data=valid_data,
      callbacks=cb
  )


171317808/171317808 [==============================] - 583s 3us/step
Epoch 1/50
100/100 [==============================] - 33s 246ms/step - loss: 0.2448 - accuracy: 0.9026 - val_loss: 0.0631 - val_accuracy: 0.9710
Epoch 2/50
100/100 [==============================] - 24s 240ms/step - loss: 0.1001 - accuracy: 0.9652 - val_loss: 0.0387 - val_accuracy: 0.9928
Epoch 3/50
100/100 [==============================] - 25s 244ms/step - loss: 0.0727 - accuracy: 0.9759 - val_loss: 0.0040 - val_accuracy: 1.0000
Epoch 4/50
100/100 [==============================] - 21s 208ms/step - loss: 0.0454 - accuracy: 0.9840 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 5/50
100/100 [==============================] - 24s 241ms/step - loss: 0.0357 - accuracy: 0.9884 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 6/50
100/100 [==============================] - 21s 209ms/step - loss: 0.0273 - accuracy: 0.9897 - val_loss: 0.0072 - val_accuracy: 1.0000
Epoch 7/50
100/100 [==============================] - 21s 208

In [13]:
model = keras.models.load_model('.\ResNet101V2-01.h5') # Training Accuracy : 99.0% Validation Accuracy : 100.0%
model.evaluate(test_data)

3/3 [==============================] - 3s 184ms/step - loss: 0.0025 - accuracy: 1.0000


[0.0024891062639653683, 1.0]